In [53]:
import numpy as np
import pandas as pd
import glob
import json
import hashlib
from datetime import timedelta

### Raw Data 불러오기

In [54]:
df = pd.read_csv('/root/workspace/JH/Uplift/data/login_logs/total.csv')
df.inDate = pd.to_datetime(df.inDate.apply(lambda x: x[:-1])) + timedelta(hours=9) # KST
df['gamer_id'] = df['gamer_id'].apply(lambda x : hashlib.sha256(x.split('_')[1].encode()).hexdigest())
df['Login'] = 1
df = df[['game_id', 'gamer_id', 'inDate', 'Login']]
df

,game_id,gamer_id,inDate,Login
0,1442,ee7e0264108d8ba198b1052eaa9f14a580145e8307c644...,2021-07-01 09:00:54.174,1
1,1442,ee7e0264108d8ba198b1052eaa9f14a580145e8307c644...,2021-07-01 09:01:02.105,1
2,2147,56a51e205fee849c8857300e426354a4402b75715f76b8...,2021-07-01 09:01:15.138,1
3,1046,8e57858ab42fef1ee804b572421beb315e9fb3bd562f10...,2021-07-01 09:03:27.895,1
4,1482,6da2374e596f5911352b528a64c9f0041f1330174c6d2b...,2021-07-01 09:06:04.362,1
...,...,...,...,...
189980,1572,0b6b2a784da2f04ac816130c772789c49623a02816c9ba...,2021-10-13 10:43:58.708,1
189981,1908,2ca9f63bb06128cdda3ab45ebc7033656c2495bbb82308...,2021-10-13 10:44:07.646,1
189982,1908,2ca9f63bb06128cdda3ab45ebc7033656c2495bbb82308...,2021-10-13 10:44:18.560,1
189983,1572,40aea607fd3cb6c43d2eb5bb7a190384539a35da90d102...,2021-10-13 10:45:53.676,1


### 데이터 만드는 함수

In [55]:
#게임-유저 별로 사용될 시간 프레임 만들기
#start=시작날짜, end='끝나는날짜', freq='몇시간/몇일 등 간격 정하기', periods='몇개를 만들건지'(end와 함께 쓸 수 없음)
def make_dateframe(start=None ,end=None, freq=None, periods=None):
    
    dates=pd.date_range(start=start ,end=end, periods=periods, freq=freq)
    str_dates=[str(d) for d in dates]
    
    #행:날짜(시간) 열:범주화한 url 2개, IAP, Login
    date_frame=pd.DataFrame(np.zeros((len(str_dates),4), dtype=int), \
                columns=['social_url', 'non_social_url', 'IAP', 'Login'], index=str_dates)
    return date_frame

### 데이터프레임, 데이터 형태 만들기

In [56]:
lag='1H' #일단 지금은 데이터가 별로 없어서 1시간 단위로 설정

#나중에 더 추가할 예정
if 'H' in lag:
    if len(lag)==2:
        hour_hop=int(lag[0])
        df['time']=df['inDate'].apply(lambda x:x.strftime('%Y-%m-%d %H:00:00'))

In [57]:
date_frame=make_dateframe(start='2021-07-01', end='2021-10-13', freq=lag)

In [58]:
#데이터프레임 예시
df.head() 

,game_id,gamer_id,inDate,Login,time
0,1442,ee7e0264108d8ba198b1052eaa9f14a580145e8307c644...,2021-07-01 09:00:54.174,1,2021-07-01 09:00:00
1,1442,ee7e0264108d8ba198b1052eaa9f14a580145e8307c644...,2021-07-01 09:01:02.105,1,2021-07-01 09:00:00
2,2147,56a51e205fee849c8857300e426354a4402b75715f76b8...,2021-07-01 09:01:15.138,1,2021-07-01 09:00:00
3,1046,8e57858ab42fef1ee804b572421beb315e9fb3bd562f10...,2021-07-01 09:03:27.895,1,2021-07-01 09:00:00
4,1482,6da2374e596f5911352b528a64c9f0041f1330174c6d2b...,2021-07-01 09:06:04.362,1,2021-07-01 09:00:00


### 형식에 맞춰서 데이터 넣기

In [59]:
#전체 게임, 유저 리스트
game_gamer_ls=list(set([(game, gamer_id) for (game, gamer_id) in zip(df.game_id, df.gamer_id)]))

#게임-유저의 시간대별 합
groupby=df.groupby(['game_id','gamer_id', 'time']).sum()[['Login']].reset_index()

In [60]:
#게임{유저:데이터프레임} 형식 만들기
for i in range(len(game_gamer_ls)):
    item=game_gamer_ls[i]
    if i==0:
        to_dict={item[0]:{item[1]:date_frame}}

    else:
        
        #game_id만 있을 때
        if item[0] in to_dict.keys():
            to_dict[item[0]].update({item[1]:date_frame})
            
        #game_id가 없을 때:
        elif item[0] not in to_dict.keys():
            to_dict[item[0]]={item[1]:date_frame}
        

In [61]:
from tqdm import tqdm
#계산한 값 위치에 맞게 넣기 
for idx in tqdm(game_gamer_ls, total=len(game_gamer_ls)):
    #행 하나씩 가져오기 (오래걸리는데 어떻게 짜야될지 모르겠어요ㅠㅠ)
    res=groupby[(groupby['game_id']==idx[0]) & (groupby['gamer_id']==idx[1])].values[0]
    #어떤 게임의 어떤 유저의 데이터프레임 가져오기
    temp=to_dict[idx[0]][idx[1]]

    #만든 날짜프레임 안에있는 데이터만 넣기 (res[2] : time)
    if res[2] in temp.index:        
        temp.loc[res[2],'Login']=res[3]
    else:
        pass 

100%|██████████| 10000/10000 [00:57<00:00, 172.94it/s]


In [62]:
to_dict[808]['db9351a297a7362b3c913ac8de77bd9a1ccc0d61bada939db15a315f5e8113fc'].sample(10)

,social_url,non_social_url,IAP,Login
2021-07-31 18:00:00,0,0,0,1
2021-08-20 22:00:00,0,0,0,4
2021-09-17 09:00:00,0,0,0,2
2021-08-21 19:00:00,0,0,0,2
2021-09-28 11:00:00,0,0,0,1
2021-09-16 18:00:00,0,0,0,2
2021-08-05 05:00:00,0,0,0,0
2021-07-20 11:00:00,0,0,0,2
2021-08-13 02:00:00,0,0,0,0
2021-10-06 22:00:00,0,0,0,2


### 저장하기

In [17]:
#json 타입으로 저장하기
#dataframe -> dictionary
out_dict={}
for idx in game_gamer_ls:
    temp=to_dict[idx[0]][idx[1]]
    dic=temp.to_dict()
    out_dict[idx[0]]={idx[1]:dic}

import json
with open('input_data_login.json','w')as f:
    json.dump(out_dict, f, indent=4)

In [18]:
#pickle 타입으로 저장하기 (용량 큼)
import pickle
with open('input_data_login_pickle','wb')as f:
    pickle.dump(out_dict, f)